In [17]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

from torch.utils.data import DataLoader
from torch.optim import Optimizer

import torch.nn.functional as F

# def loss_func(out, y):
#     return F.binary_cross_entropy_with_logits(out, y)

loss_func = nn.CrossEntropyLoss()

def train_one_epoch(model: nn.Module, optimizer: Optimizer, data_loader: DataLoader, device, epoch):
    model.train()
    model.zero_grad()
    tqdm_dataloader = tqdm(data_loader)
    total_batch = 0
    for targets in tqdm_dataloader:
        images = targets['image'].to(device)
        labels = targets['label'].to(device)
        total_batch += len(images)

        optimizer.zero_grad()
        out = model(images)

        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()

        tqdm_dataloader.set_description(
            f"Epoch {epoch + 1}, lr is {optimizer.param_groups[0]['lr']:.6f} loss {loss.item():.3f}")

In [2]:
from PIL import Image
from torchvision import transforms

from torch.utils.data import Dataset

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)


class TrainDataset_01(Dataset):
    def __init__(self, img_paths, labels, transform = None):
        self.img_paths = img_paths
        self.labels = labels
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((512, 384), Image.BILINEAR),
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
            ])
        else:
            self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)
        return {'image': image, 'label': label}
    
    def __len__(self):
        return len(self.img_paths)

In [3]:
import os
import pandas as pd

In [4]:
import torchvision

In [5]:
imagenet_resnet152 = torchvision.models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /opt/ml/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


In [6]:
print(imagenet_resnet152)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
imagenet_resnet152.fc = nn.Linear(in_features=2048, out_features=18, bias=True)
nn.init.xavier_uniform_(imagenet_resnet152.fc.weight)
imagenet_resnet152.fc.bias.data.uniform_(-0.01, 0.01)

tensor([ 0.0007, -0.0011, -0.0045,  0.0058, -0.0082, -0.0089, -0.0065, -0.0090,
        -0.0093, -0.0073, -0.0004,  0.0010,  0.0091,  0.0097,  0.0046,  0.0023,
         0.0020, -0.0003])

In [12]:
EPOCH = 10
lr = 0.001
device = torch.device('cuda')

TRAIN_DIR = '/opt/ml/input/data/train'
train_info = pd.read_csv(os.path.join(TRAIN_DIR, 'train_info_01.csv'))

image_paths = train_info['path'].values
labels = train_info['label'].values

In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()


In [19]:
from torch.optim import Adam
optimizer = Adam(imagenet_resnet152.parameters(), lr=lr)
train_dataset = TrainDataset_01(image_paths, labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [21]:
imagenet_resnet152.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
train_one_epoch(imagenet_resnet152, optimizer, train_loader, device, 1)

In [31]:
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
test_image_dir = os.path.join(test_dir, 'images')
test_image_paths = [os.path.join(test_image_dir, img_id) for img_id in submission.ImageID]

In [33]:
transform = transforms.Compose([
    transforms.Resize((512, 384), Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(test_image_paths, transform)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [34]:
imagenet_resnet152.eval()
all_predictions = []
tqdm_test_dataloader = tqdm(test_loader)
for images in tqdm_test_dataloader:
    with torch.no_grad():
        images = images.to(device)
        pred = imagenet_resnet152(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

In [35]:
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,14
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,10
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,16
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0


In [36]:
submission.to_csv('submission_test.csv', index=False)